# Importing all modules

In [ ]:
import PIL
from PIL import Image
from PIL import ImageFont
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageOps

from pathlib import Path
import numpy as np 
import pandas as pd
import cv2
import seaborn as sns


In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

In [ ]:
# Just to see what file is available is the directory
# fastai has more detailed implementation of ls
Path.ls = lambda x: list(x.iterdir())

In [ ]:
path = Path(r'/kaggle/input/vinbigdata-chest-xray-abnormalities-detection')
train_dir = Path(path/'train')
test_dir = Path(path/'test')

# Investigating `train_df`

In [ ]:
len(train_dir.ls()), len(test_dir.ls())

In [ ]:
train_df = pd.read_csv(path/'train.csv')
train_df.head()

In [ ]:
train_df.info()

In [ ]:
train_df['class_name'].value_counts().plot.barh()

## Without No finding column total number of examples in training data set

In [ ]:
train_df.loc[train_df['class_name'] != 'No finding','class_name'].value_counts().plot.barh()

In [ ]:
test_images_number = 2
random_images = np.random.choice(train_dir.ls(), test_images_number)

# Reading pydicom image and converting them three channel PIL image

In [ ]:
import pydicom
from pydicom import dcmread

In [ ]:
ds = dcmread(random_images[0])
ds

In [ ]:
ds.PatientSex

In [ ]:
image_size = ((ds.Rows),(ds.Columns))
image_size

Code also taken from [this Notebook](https://www.kaggle.com/bjoernholzhauer/eda-dicom-reading-vinbigdata-chest-x-ray) and also combines code from 
[here](https://github.com/vinbigdata-medical/vindr-cxr/blob/main/outlier-detection/transforms.py)

In [ ]:
def from_dicom_to_3_channel_numpy(file_name):
    """
    convert dicoḿ file to 3 channel numpy array 
    and also human readable format
    
    file_name = name of the dicom file
    
    3 images will be plotted as an output
    first image = after reading dicom files
    second image = after converting to human readable format 
    last image = three channel image
    
    image : return 3 channel image
    """
    dcm_file = pydicom.dcmread(file_name)
  
    if dcm_file.BitsStored in (10, 12):
        dcm_file.BitsStored = 14
    
    raw_image = dcm_file.pixel_array
    raw_image = pydicom.pixel_data_handlers.util.apply_voi_lut(raw_image , dcm_file)

    # Normalize pixel to be in [0, 255]
    rescaled_image = cv2.convertScaleAbs(raw_image,
                                         alpha=(255.0/raw_image.max()))
    # Correct image inversion
    if dcm_file.PhotometricInterpretation == "MONOCHROME1":
        rescaled_image = cv2.bitwise_not(rescaled_image)
    
    # Perform histogram equalization if the input is 
    # original dicom file

    if raw_image.max() > 255:
        adjusted_image = cv2.equalizeHist(rescaled_image)
    else:
        adjusted_image = rescaled_image

    image = Image.fromarray(adjusted_image)
    image = image.convert('RGB')
    
    
    # plotting all 3 main converions done in this function
    plt.figure(figsize=(18, 5))
    
    plt.subplot(1,3,1)
    plt.imshow(raw_image, cmap='gray', label='dicom file')
    plt.axis('off')
    plt.subplot(1,3,2)
    plt.imshow(rescaled_image, cmap='gray', label = 'human readable')
    plt.axis('off')
    plt.subplot(1,3,3)
    plt.imshow(image, cmap='gray', label = 'Three channel')
   
    plt.axis('off')
    plt.subplots_adjust()
    plt.legend()
    return image

In [ ]:
image = from_dicom_to_3_channel_numpy(file_name = random_images[1])

# let me try bounding box plotting


As No finding have no bounding box therefore removing no founding 

In [ ]:
classified_df = train_df.loc[train_df['class_name'] != 'No finding',:]

In [ ]:
classified_df.head()

In [ ]:
total_images_available = list(set(classified_df['image_id']))

## How much bounding box availble per image

In [ ]:
# only see how much unique image and total image availble
len(total_images_available),classified_df.shape

In [ ]:
bbox_per_image = {grp:data.shape[0] for  grp, data in classified_df.groupby('image_id')}
 
    
print(f'Maximum box number per image  = {np.max(list(bbox_per_image.values()))},\nMinimum box number= {np.min(list(bbox_per_image.values()))}, \nAverage box number  {np.round(np.average(list(bbox_per_image.values())))}')

In [ ]:
plt.plot(list(bbox_per_image.values()))

## Now plotting a sample which has relative smaller bounding box

our previous function plot 3 images. that was only for observation. We 
actually need a  function which just return the pil image. `dicom_to_pil` is the same fuction as previous.
Just without the plotting the image

In [ ]:
def dicom_to_pil(file_name):
    """
    convert dicoḿ file to 3 channel numpy array 
    and also human readable format
    
    file_name = name of the dicom file 
    image : return 3 channel image
    """
    dcm_file = pydicom.dcmread(file_name)
  
    if dcm_file.BitsStored in (10, 12):
        dcm_file.BitsStored = 16
    
    raw_image = dcm_file.pixel_array
    raw_image = pydicom.pixel_data_handlers.util.apply_voi_lut(raw_image , dcm_file)

    # Normalize pixel to be in [0, 255]
    rescaled_image = cv2.convertScaleAbs(raw_image,
                                         alpha=(255.0/raw_image.max()))
    # Correct image inversion
    if dcm_file.PhotometricInterpretation == "MONOCHROME1":
        rescaled_image = cv2.bitwise_not(rescaled_image)
    
    # Perform histogram equalization if the input is 
    # original dicom file

    if raw_image.max() > 255:
        adjusted_image = cv2.equalizeHist(rescaled_image)
    else:
        adjusted_image = rescaled_image

    image = Image.fromarray(adjusted_image)
    image = image.convert('RGB')
    return image

In [ ]:
sample = '9a5094b2563a1ef3ff50dc5c7ff71345'
sample_box = classified_df.loc[classified_df['image_id'] == sample, :]
sample_box

In [ ]:
base_file = train_dir
file_name = f'{base_file}/{sample}.dicom'
image = dicom_to_pil(file_name)

In [ ]:
# let me see the image first
plt.figure(figsize=(18, 5))
plt.imshow(image)
plt.axis('off');

## Now bounding box 

I recently completer one coursera online course [Advanced Computer vision with tensorflow](https://www.coursera.org/learn/advanced-computer-vision-with-tensorflow/home/welcome), where a nice utility function was available, where if you give bounding box and image it will create 
bounding box. There are different other nice function was availble. One can easily look at the course

In [ ]:
def draw_bounding_box_on_image(image,
                               ymin,
                               xmin,
                               ymax,
                               xmax,
                               color='red',
                               thickness=1,
                               display_str=[],
                               use_normalized_coordinates=True):
    """
    Adds a bounding box to an image.
    Bounding box coordinates can be specified in either absolute (pixel) or
    normalized coordinates by setting the use_normalized_coordinates argument.
    Args:
        image: a PIL.Image object.
        ymin: ymin of bounding box.
        xmin: xmin of bounding box.
        ymax: ymax of bounding box.
        xmax: xmax of bounding box.
        color: color to draw bounding box. Default is red.
        thickness: line thickness. Default value is 4.
        display_str_list: string to display in box
        use_normalized_coordinates: If True (default), treat coordinates
          ymin, xmin, ymax, xmax as relative to the image.  Otherwise treat
    coordinates as absolute.
    """
    draw = PIL.ImageDraw.Draw(image)
    im_width, im_height = image.size
    if use_normalized_coordinates:
        (left, right, top, bottom) = (xmin*im_width, xmax*im_width, ymin*im_height, ymax*im_height)
    else:
        (left, right, top, bottom) = (xmin, xmax, ymin, ymax)
    draw.line([(left, top), (left, bottom), (right, bottom),
             (right, top), (left, top)], width=thickness, fill=color)
    
    
    if len(display_str) >= 1:
        try:
            font = ImageFont.truetype("/usr/share/fonts/truetype/liberation/LiberationSansNarrow-Regular.ttf",
                                  50)
        except IOError:
            print("Font not found, using default font.")
            font = ImageFont.load_default()


        # If the total height of the display strings added to the top of the bounding
        # box exceeds the top of the image, stack the strings below the bounding box
        # instead of above.
        display_str_heights = [font.getsize(ds)[1] for ds in display_str]
        # Each display_str has a top and bottom margin of 0.05x.
        total_display_str_height = (1 + 2 * 0.05) * sum(display_str_heights)

        if top > total_display_str_height:
            text_bottom = top
        else:
            text_bottom = top + total_display_str_height

        text_width, text_height = font.getsize(display_str[0])
        margin = np.ceil(0.05 * text_height)
        draw.rectangle([(left, text_bottom - text_height - 2 * margin),
                        (left + text_width, text_bottom)],
                       fill=color)
        draw.text((left + margin, text_bottom - text_height - margin),
                  display_str[0],
                  fill="black",
                  font=font)
        text_bottom -= text_height - 2 * margin


In [ ]:
def bounding_boxes_in_pil_image(sample):
    """
    
    """
    
    sample_box = classified_df.loc[classified_df['image_id'] == sample, :]
    base_file = train_dir
    file_name = f'{base_file}/{sample}.dicom'
    image = dicom_to_pil(file_name)
    image_pil = (image)
    rgbimg = PIL.Image.new("RGBA", image_pil.size)
    rgbimg.paste(image_pil)
    for i in sample_box.index:
    # creating bounding box for each row
        x_min, y_min, x_max, y_max = sample_box.loc[i, ['x_min','y_min','x_max', 'y_max']]
        draw_bounding_box_on_image(image=rgbimg,
                          ymin=y_min,
                          xmin=x_min,
                          ymax=y_max,
                          xmax=x_max,
                          color='red',
                          thickness=10,
                          use_normalized_coordinates=False)
    image_bbox = np.array(rgbimg)
    plt.figure(figsize=(18, 12))
    plt.imshow(image_bbox)
    plt.axis('off')
    

In [ ]:
bounding_boxes_in_pil_image(sample)

## let's plot extreme boxes. At first we need to search for the sample has 57 boxes

In [ ]:
{key for key, value in bbox_per_image.items() if value == 57}

In [ ]:
new_sample = '03e6ecfa6f6fb33dfeac6ca4f9b459c9'
bounding_boxes_in_pil_image(new_sample)

Why so much bounding boxes
 - different detectors so diffrent places
 - it has different problems that's why so much boxes

First case ís availble it is sure. Let me see whether second case is availble 
We need to change the colours. At first based on problems and then different doctors


### first define colors for the Radiologist


In [ ]:
# I want sort the radiologist. Therefore sorted function key
# would be then the number and `func_name` just search for it 
func_name = lambda x: int(x[1:])

# we have a sorted list for Radiologist now
all_radiologist_list = sorted(list(set(classified_df['rad_id'])), key= func_name)

# searching for the color palltte from seaborn and 
# colors will be the number of total radiologist
colors = sns.color_palette(None, len(all_radiologist_list))

# creating a dictionary where key will be radiologist and 
# value will be the color
rediologist_color_map = {key: color for key, color in zip(all_radiologist_list, colors)}
colors


We need to change the function where we create the bounding in a sample image

In [ ]:
def bounding_boxes_in_pil_image(sample:str,
                               color_map=rediologist_color_map,
                               maping='rad_id'):
    """
    creating a bounding box in the image 
    sample: image id 
    color_map : a dictionary key is the color group e.g. Radiologist, value is the 
    color name 
    maping:str: which coloumn needs to be mapped in case of radiologist it is rad_id
    """
    
    sample_box = classified_df.loc[classified_df['image_id'] == sample, :]
    base_file = train_dir
    file_name = f'{base_file}/{sample}.dicom'
    image = dicom_to_pil(file_name)
    image_pil = (image)
    rgbimg = PIL.Image.new("RGBA", image_pil.size)
    rgbimg.paste(image_pil)
    for i in sample_box.index:
    # creating bounding box for each row
        x_min, y_min, x_max, y_max = sample_box.loc[i, ['x_min','y_min','x_max', 'y_max']]
        class_clr = sample_box.loc[i, maping]
        clr_id = color_map[sample_box.loc[i, maping]]
   
        color = int(clr_id[0]*255), int(clr_id[1]*255), int(clr_id[2]*255)

        draw_bounding_box_on_image(image=rgbimg,
                          ymin=y_min,
                          xmin=x_min,
                          ymax=y_max,
                          xmax=x_max,
                          color=color,
                          thickness=10,
                          display_str = [class_clr],
                          use_normalized_coordinates=False)
    image_bbox = np.array(rgbimg)
    

  
    
    plt.figure(figsize=(18, 12))
    plt.imshow(image_bbox)
    plt.axis('off')

In [ ]:
{key for key, value in bbox_per_image.items() if value == 57}
new_sample = '03e6ecfa6f6fb33dfeac6ca4f9b459c9'
bounding_boxes_in_pil_image(new_sample)

### Now we need to see for each class 

In [ ]:
class_name_list = sorted(list(set(classified_df['class_name'])))

class_name_color = sns.color_palette(None, len(class_name_list))
class_name_color_map = {key: color for key, color in zip(class_name_list, class_name_color)}
class_name_color

In [ ]:
bounding_boxes_in_pil_image(new_sample,  color_map=class_name_color_map,
                               maping='class_name')

# Box Fusion

Normally to to remmove overlapping area one technique called `Non-max Supression`. [here](https://paperswithcode.com/method/non-maximum-suppression) is nice explanation

- We will go through all the images 
  - if more than one radiologist is avilable we will see iou accuracy 
  - if more than a threshold then may be same area so we need to somehow take only one not three
      - need to take one base box don't know may be see the data
      


In [ ]:
new_sample = '011244ab511b20130d846f5f8f0c3866'
bounding_boxes_in_pil_image(new_sample,
                            color_map = rediologist_color_map,
                            maping='rad_id')

After analysis I have found that each image consists of Three radiologist bounding box.
- If each radiologist has only one box then I will take the bounding box which covers all radiologist area
- If one radioligist has more bounding box then the radiologist who has more bounding boxes will only be chosen and others will be removed

First start simple. look only the images which has only three bounding boxes, means one bounding box per radiologist

In [ ]:
simple_image_id = [grp for grp, data in classified_df.groupby('image_id') if len(data) == 3]
len(simple_image_id)

We have 370 images which has only one box per radiologist


let's try to see the image

In [ ]:
def fused_box_in_image(sample,x_min,
                      y_min, x_max, 
                      y_max,color_map=rediologist_color_map,
                      maping='rad_id', all=True):
    """
    sample: sample image id
    x_min: combined box x_min
    y_min: combined box y_min
    x_max: combined box x_max
    x_min: combined box y_min
    color_map : a color maping of seaborn based on number of unique instance:
    here it is number of radiologist
    
    maping: which column will be mapped based on color maping
    
    2 Column image first one will be combined bounding box 
    second one will be with both combined image and all radiologist detection
    """
    
    sample_box = classified_df.loc[classified_df['image_id'] == sample, :]
    base_file = train_dir
    file_name = f'{base_file}/{sample}.dicom'
    image = dicom_to_pil(file_name)
    image_pil = (image)
    rgbimg = PIL.Image.new("RGBA", image_pil.size)
    
    rgbimg.paste(image_pil)
    plt.figure(figsize=(18, 12))
    plt.subplot(121)
    
    draw_bounding_box_on_image(image=rgbimg,
                          ymin=y_min,
                          xmin=x_min,
                          ymax=y_max,
                          xmax=x_max,
                          color='red',
                          thickness=10,
                          display_str = ['combined box'],
                          use_normalized_coordinates=False)
    plt.imshow(np.array(rgbimg))
    plt.axis('off')
    if all:
        for i in sample_box.index:
        # creating bounding box for each row
            x_min, y_min, x_max, y_max = sample_box.loc[i, ['x_min','y_min','x_max', 'y_max']]
            class_clr = sample_box.loc[i, maping]
            clr_id = color_map[sample_box.loc[i, maping]]

            color = int(clr_id[0]*255), int(clr_id[1]*255), int(clr_id[2]*255)

            draw_bounding_box_on_image(image=rgbimg,
                              ymin=y_min,
                              xmin=x_min,
                              ymax=y_max,
                              xmax=x_max,
                              color=color,
                              thickness=10,
                              display_str = [class_clr],
                              use_normalized_coordinates=False)
    image_bbox = np.array(rgbimg)
    plt.subplot(122)
    plt.imshow(image_bbox)
    plt.axis('off')

In [ ]:
sample_df = classified_df.loc[classified_df['image_id'] == simple_image_id[0],:]

In [ ]:
x_min_n, y_min_n, x_max_n, y_max_n = np.min(sample_df['x_min']),np.min(sample_df['y_min']),np.max(sample_df['x_max']),np.max(sample_df['y_max'])

In [ ]:
fused_box_in_image(sample=simple_image_id[0],
                   x_min=x_min_n,
                   y_min=y_min_n, 
                   x_max=x_max_n,
                   y_max=y_max_n,all=True)

# We have done it !! or not :) see the next part

In [ ]:
simple_images = classified_df.loc[classified_df['image_id'].isin(simple_image_id),:]

In [ ]:
combined_df = {}
for grp, data in simple_images.groupby('image_id'):
    class_names = list(set(data['class_name']))
    if len(class_names) == 1:
        x_min_n, y_min_n, x_max_n, y_max_n = np.min(data['x_min']),np.min(data['y_min']),np.max(data['x_max']),np.max(data['y_max'])
        combined_df[grp]=  x_min_n, y_min_n, x_max_n, y_max_n, data['class_name']
    elif len(class_names) == 3:
        bounding_boxes_in_pil_image(grp,
                            color_map = rediologist_color_map,
                            maping='rad_id')
        

In [ ]:
So Actually it was not so eassy that we will combine alltogether
- some are easy but different radiologists sometimes detect different deasees
- Need further investigation

In [ ]:
U

# Next Step


- Somehow create common bounding area for different Radiologist
-  try some augmentation [albumenation] on the image
- See that augmentation on a image 
- Apply bounding box to the image
- Apply Augmentation to the image and label together
- See how look like the augmentation with bounding box in the image
- create unet model or load a model with pretrained weight 
- see the base modle cosine annealing implementation
- see tensorflow course what they have done
   - training
   - callback
- Submit atleast one submission